In [2]:
# -*- coding: utf-8 -*-
"""
Generador de datos demo de People Analytics y exportación a CSV.

Crea ficheros CSV con las mismas estructuras y volumen aproximado
que el script original orientado a SQL Server.

Autor: ChatGPT — Junio 2025
"""

import os
import csv
import random
from datetime import datetime, timedelta, date

import pandas as pd
from faker import Faker

fake = Faker("es_ES")

# ----------------------- CONFIGURACIÓN -----------------------
NUM_TIENDAS = 5
NUM_PUESTOS = 8
NUM_EMPLEADOS = 1000
FICHAJES_BATCH = 10_000  # número de filas a escribir por lote
START_FICH = date(2024, 1, 1)
END_FICH = date(2025, 12, 31)
OUTPUT_DIR = "output_csv"  # carpeta donde se guardarán los CSV

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------------- UTILS ---------------------------

def daterange(d1: date, d2: date):
    """Generador de fechas, ambos extremos incluidos."""
    for n in range((d2 - d1).days + 1):
        yield d1 + timedelta(days=n)

# -------------------- 1. TABLAS MAESTRAS --------------------

def build_tiendas():
    rows = []
    for i in range(1, NUM_TIENDAS + 1):
        rows.append({
            "ID_Tienda": i,
            "Nombre_Tienda": f"Tienda {i} {fake.city_suffix()}",
            "Ciudad": fake.city(),
            "Provincia": fake.state(),
            "Pais": "España",
            "Fecha_Apertura": fake.date_between(start_date="-4000d", end_date="-365d"),
            "Metros_Cuadrados": random.randint(400, 1500),
        })
    return rows


def build_puestos():
    puestos_demo = [
        ("Dependiente/a", "Ventas", "Operativo"),
        ("Encargado/a", "Ventas", "Mando intermedio"),
        ("Cajero/a", "Operaciones", "Operativo"),
        ("Visual merchandiser", "Marketing", "Especialista"),
        ("Responsable RR. HH.", "RRHH", "Mando medio"),
        ("Almacén", "Logística", "Operativo"),
        ("Gerente tienda", "Dirección tienda", "Manager"),
        ("Personal limpieza", "Servicios", "Operativo"),
    ]
    return [
        {
            "ID_Puesto": i,
            "Nombre_Puesto": nombre,
            "Departamento": dept,
            "Nivel": nivel,
        }
        for i, (nombre, dept, nivel) in enumerate(puestos_demo, start=1)
    ]

# ----------------------- 2. EMPLEADOS -----------------------

def build_empleados():
    rows = []
    for i in range(1, NUM_EMPLEADOS + 1):
        puesto_id = random.randint(1, NUM_PUESTOS)
        tienda_id = random.randint(1, NUM_TIENDAS)
        fecha_alta = fake.date_between(start_date="-2500d", end_date="-30d")
        jornada = random.choices(["Completa", "Parcial"], weights=[0.8, 0.2])[0]
        contrato = random.choices(["Indefinido", "Temporal", "Prácticas"], weights=[0.65, 0.3, 0.05])[0]
        salario = random.randint(18_000, 32_000)
        rows.append({
            "ID_Empleado": i,
            "Nombre": fake.first_name(),
            "Apellido": fake.last_name(),
            "Genero": random.choice(["M", "F", "O"]),
            "Fecha_Nacimiento": fake.date_between(start_date="-60y", end_date="-18y"),
            "Fecha_Alta": fecha_alta,
            "Estado": "Activo",
            "ID_Puesto": puesto_id,
            "ID_Tienda": tienda_id,
            "Tipo_Contrato": contrato,
            "Jornada": jornada,
            "Salario_Base": salario,
        })
    return rows

# ------------------ 3. FICHAJES (MUY GRANDES) ---------------

def write_fichajes_csv(filepath: str):
    """Genera el CSV de Fichajes escribiendo en streaming."""
    header = ["ID_Fichaje", "ID_Empleado", "Fecha_Hora", "Tipo", "Dispositivo"]
    with open(filepath, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(header)

        fichaje_id = 1
        batch = []
        for emp_id in range(1, NUM_EMPLEADOS + 1):
            base_start = fake.random_int(min=7, max=11)  # 7‑11h de inicio
            for day in daterange(START_FICH, END_FICH):
                # Salta algunos fines de semana (tiendas abren sábados ~50 %)
                if day.weekday() >= 5 and random.random() < 0.5:
                    continue

                entrada = datetime.combine(day, datetime.min.time()) + timedelta(
                    hours=base_start, minutes=random.randint(-15, 15)
                )
                salida = entrada + timedelta(hours=8, minutes=random.randint(-10, 20))

                batch.append([fichaje_id, emp_id, entrada, "Entrada", "terminal-01"])
                fichaje_id += 1
                batch.append([fichaje_id, emp_id, salida, "Salida", "terminal-01"])
                fichaje_id += 1

                if len(batch) >= FICHAJES_BATCH:
                    writer.writerows(batch)
                    batch.clear()

        # Último lote
        if batch:
            writer.writerows(batch)

    print(f"Fichajes CSV generado: {filepath}")

# ------------------ 4. DATOS COMPLEMENTARIOS ---------------

def build_ausencias(num_reg=800):
    rows = []
    for i in range(1, num_reg + 1):
        emp_id = random.randint(1, NUM_EMPLEADOS)
        inicio = fake.date_between(start_date="-365d", end_date="today")
        fin = inicio + timedelta(days=random.randint(1, 5))
        tipo = random.choices(
            ["Enfermedad", "Vacaciones", "Permiso", "No justificada"],
            weights=[0.5, 0.3, 0.15, 0.05],
        )[0]
        rows.append({
            "ID_Ausencia": i,
            "ID_Empleado": emp_id,
            "Fecha_Inicio": inicio,
            "Fecha_Fin": fin,
            "Tipo_Ausencia": tipo,
            "Comentario": fake.sentence(nb_words=6),
        })
    return rows


def build_movimientos(num_reg=120):
    motivos = ["Alta", "Baja", "Cambio_Puesto", "Traslado_Tienda"]
    rows = []
    for i in range(1, num_reg + 1):
        emp_id = random.randint(1, NUM_EMPLEADOS)
        fecha = fake.date_between(start_date="-730d", end_date="today")
        tipo = random.choice(motivos)
        detalle = fake.sentence(nb_words=8)
        rows.append({
            "ID_Movimiento": i,
            "ID_Empleado": emp_id,
            "Fecha_Mov": fecha,
            "Tipo_Mov": tipo,
            "Detalle": detalle,
        })
    return rows


def build_evaluaciones(num_reg=1500):
    rows = []
    for i in range(1, num_reg + 1):
        emp_id = random.randint(1, NUM_EMPLEADOS)
        fecha = fake.date_between(start_date="-730d", end_date="today")
        puntuacion = round(random.uniform(2.5, 5.0), 2)
        rows.append({
            "ID_Evaluacion": i,
            "ID_Empleado": emp_id,
            "Fecha_Eval": fecha,
            "Puntuacion": puntuacion,
            "Comentarios": fake.sentence(nb_words=5),
            "Evaluador": fake.name(),
        })
    return rows


def build_formaciones(num_reg=900):
    cursos = [
        "Moda Sostenible",
        "Atención al cliente",
        "Visual Merchandising",
        "Gestión de Inventario",
        "Liderazgo de equipos",
        "Prevención Riesgos",
    ]
    rows = []
    for i in range(1, num_reg + 1):
        emp_id = random.randint(1, NUM_EMPLEADOS)
        curso = random.choice(cursos)
        inicio = fake.date_between(start_date="-730d", end_date="-10d")
        fin = inicio + timedelta(days=random.choice([1, 2, 3, 5]))
        horas = random.randint(4, 24)
        rows.append({
            "ID_Formacion": i,
            "ID_Empleado": emp_id,
            "Curso": curso,
            "Fecha_Inicio": inicio,
            "Fecha_Fin": fin,
            "Horas": horas,
            "Certificado": f"Certificado {curso}",
        })
    return rows


def build_costes_salariales():
    rows = []
    coste_id = 1
    for emp_id in range(1, NUM_EMPLEADOS + 1):
        for mes in range(18):  # 18 meses retrospectivos
            ref_mes = datetime.today().replace(day=1) - timedelta(days=30 * mes)
            rows.append({
                "ID_Coste": coste_id,
                "ID_Empleado": emp_id,
                "Mes": ref_mes.date(),
                "Salario_Fijo": random.randint(1200, 2200),
                "Variables": random.randint(0, 400),
                "Seguridad_Social": random.randint(400, 650),
            })
            coste_id += 1
    return rows

# --------------------- FUNCIONES AUXILIARES ------------------

def save_dataframe(rows, filename):
    """Persistir una lista de diccionarios a CSV."""
    df = pd.DataFrame(rows)
    path = os.path.join(OUTPUT_DIR, filename)
    df.to_csv(path, index=False)
    print(f"{filename} creado con {len(df):,} filas.")

# ---------------------- EJECUCIÓN MAIN ----------------------

def main():
    print("Generando tablas maestras…")
    save_dataframe(build_tiendas(), "Tiendas.csv")
    save_dataframe(build_puestos(), "Puestos.csv")

    print("Generando empleados…")
    save_dataframe(build_empleados(), "Empleados.csv")

    print("Generando fichajes (puede tardar)…")
    write_fichajes_csv(os.path.join(OUTPUT_DIR, "Fichajes.csv"))

    print("Generando datos complementarios…")
    save_dataframe(build_ausencias(), "Ausencias.csv")
    save_dataframe(build_movimientos(), "Movimientos_Laborales.csv")
    save_dataframe(build_evaluaciones(), "Evaluaciones.csv")
    save_dataframe(build_formaciones(), "Formaciones.csv")
    save_dataframe(build_costes_salariales(), "Costes_Salariales.csv")

    print("\n¡Datos de People Analytics exportados correctamente a CSV en la carpeta 'output_csv'!")


if __name__ == "__main__":
    main()


Generando tablas maestras…
Tiendas.csv creado con 5 filas.
Puestos.csv creado con 8 filas.
Generando empleados…
Empleados.csv creado con 1,000 filas.
Generando fichajes (puede tardar)…
Fichajes CSV generado: output_csv\Fichajes.csv
Generando datos complementarios…
Ausencias.csv creado con 800 filas.
Movimientos_Laborales.csv creado con 120 filas.
Evaluaciones.csv creado con 1,500 filas.
Formaciones.csv creado con 900 filas.
Costes_Salariales.csv creado con 18,000 filas.

¡Datos de People Analytics exportados correctamente a CSV en la carpeta 'output_csv'!


In [1]:
import csv
import random
from faker import Faker

fake = Faker()
NUM_EMPLEADOS = 1000  # O el número que corresponda


def insert_encuestas_clima_csv(num_reg=1000):
    with open("encuestas_clima.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["ID_Encuesta", "ID_Empleado", "Fecha", "Puntuacion_Global", "Comentario"])
        for i in range(1, num_reg + 1):
            emp_id = random.randint(1, NUM_EMPLEADOS)
            fecha = fake.date_between(start_date='-730d', end_date='today')
            puntuacion = round(random.uniform(2.5, 5.0), 2)
            comentario = fake.sentence(nb_words=6)
            writer.writerow([i, emp_id, fecha, puntuacion, comentario])


def insert_candidatos_csv(num_reg=500):
    puestos = ["Dependiente/a", "Encargado/a", "Cajero/a", "Almacén", "Visual merchandiser", "Gerente tienda"]
    fuentes = ["LinkedIn", "Web", "Referido", "Portal Empleo", "Feria empleo"]
    estados = ["Entrevistado", "Rechazado", "Contratado"]

    with open("candidatos.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["ID_Candidato", "Nombre", "Fecha_Aplicacion", "Puesto_Aplicado", "Estado", "Fuente", "Evaluacion"])
        for i in range(1, num_reg + 1):
            nombre = fake.name()
            fecha = fake.date_between(start_date='-365d', end_date='today')
            puesto = random.choice(puestos)
            estado = random.choices(estados, weights=[0.4, 0.4, 0.2])[0]
            fuente = random.choice(fuentes)
            evaluacion = round(random.uniform(2.0, 5.0), 2)
            writer.writerow([i, nombre, fecha, puesto, estado, fuente, evaluacion])

def insert_feedbacks_csv(num_reg=800):
    tipos = ["Peer", "1:1", "Cliente Interno"]

    with open("feedbacks.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["ID_Feedback", "ID_Empleado", "Fecha", "Tipo", "Comentario", "Evaluador"])
        for i in range(1, num_reg + 1):
            emp_id = random.randint(1, NUM_EMPLEADOS)
            fecha = fake.date_between(start_date='-365d', end_date='today')
            tipo = random.choice(tipos)
            comentario = fake.sentence(nb_words=7)
            evaluador = fake.name()
            writer.writerow([i, emp_id, fecha, tipo, comentario, evaluador])
def insert_reconocimientos_sanciones_csv(num_reg=400):
    tipos = ["Reconocimiento", "Sanción"]
    categorias_reco = ["Trabajo en equipo", "Liderazgo", "Atención al cliente", "Innovación"]
    categorias_sanc = ["Faltas leves", "Retrasos", "Conducta inadecuada"]

    with open("reconocimientos_sanciones.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["ID_Evento", "ID_Empleado", "Fecha_Evento", "Tipo_Evento", "Descripcion", "Categoria"])
        for i in range(1, num_reg + 1):
            emp_id = random.randint(1, NUM_EMPLEADOS)
            fecha = fake.date_between(start_date='-365d', end_date='today')
            tipo = random.choice(tipos)
            categoria = random.choice(categorias_reco if tipo == "Reconocimiento" else categorias_sanc)
            descripcion = fake.sentence(nb_words=8)
            writer.writerow([i, emp_id, fecha, tipo, descripcion, categoria])


In [2]:
print("Generando archivos CSV de encuestas, candidatos, feedbacks y eventos disciplinarios…")
insert_encuestas_clima_csv()
insert_candidatos_csv()
insert_feedbacks_csv()
insert_reconocimientos_sanciones_csv()
print("¡Archivos CSV generados exitosamente!")


Generando archivos CSV de encuestas, candidatos, feedbacks y eventos disciplinarios…
¡Archivos CSV generados exitosamente!
